# **Install dependencies**

In [3]:
%pip install -q streamlit transformers==4.46.0 protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 72.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 91.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.2 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
o

# **Helpers cho Streamlit demo**

In [6]:
%%writefile helpers.py
import os
import pickle
import joblib
import numpy as np
import streamlit as st

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_ROOT = "/kaggle/input/toxic-comment-classification-models/tensorflow2/default/1"

ML_DIR = os.path.join(MODEL_ROOT, "ml")
DL_DIR = os.path.join(MODEL_ROOT, "dl")

# BiLSTM + GRU + BiGRU model
LSTM_MULTI_ARTIFACTS = os.path.join(DL_DIR, "lstm_multilabel_artifacts.pkl")
LSTM_MULTI_MODEL     = os.path.join(DL_DIR, "lstm_multilabel_word2vec.h5")

GRU_BIN_ARTIFACTS = os.path.join(DL_DIR, "gru_binary_artifacts.pkl")
GRU_BIN_MODEL     = os.path.join(DL_DIR, "gru_binary_word2vec.h5")

BIGRU_TOKENIZER_PATH = os.path.join(DL_DIR, "tokenizer_bigru.pkl")
BIGRU_MODEL_PATH     = os.path.join(DL_DIR, "bigru_fasttext.h5")
MAX_LEN_BIGRU        = 220  

# RoBERTa 
ROBERTA_DIR = os.path.join(DL_DIR, "roberta_multi")

# Label names 
LABEL_COLS = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate",
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Text preprocessing
def preprocess_text(text: str) -> str:
    """Light text cleaning"""
    import re
    text = str(text).lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# 2. Load models

# ---- ML MODELS (joblib) ----
@st.cache_resource
def load_binary_ml_model(model_name: str):
    if model_name == "Logistic Regression":
        fname = "lr_binary.joblib"
    elif model_name == "SVM":
        fname = "svm_binary.joblib"
    elif model_name == "Naive Bayes":
        fname = "nb_binary.joblib"
    else:
        raise ValueError(f"Unknown binary ML model: {model_name}")

    path = os.path.join(ML_DIR, fname)
    model = joblib.load(path)
    return model

@st.cache_resource
def load_multi_ml_model(model_name: str):
    if model_name == "Logistic Regression":
        fname = "lr_multilabel.joblib"
    elif model_name == "SVM":
        fname = "svm_multilabel.joblib"
    elif model_name == "Naive Bayes":
        fname = "nb_multilabel.joblib"
    else:
        raise ValueError(f"Unknown multi-label ML model: {model_name}")

    path = os.path.join(ML_DIR, fname)
    model = joblib.load(path)
    return model

# ---- KERAS DEEP MODELS ----
@st.cache_resource
def load_gru_binary():
    with open(GRU_BIN_ARTIFACTS, "rb") as f:
        art = pickle.load(f)

    tokenizer = art.get("tokenizer")
    max_len = art.get("max_len", art.get("maxlen", 220))

    model = tf.keras.models.load_model(GRU_BIN_MODEL, compile=False)
    return model, tokenizer, max_len

@st.cache_resource
def load_lstm_multilabel():
    with open(LSTM_MULTI_ARTIFACTS, "rb") as f:
        art = pickle.load(f)

    tokenizer = art.get("tokenizer")
    max_len = art.get("max_len", art.get("maxlen", 220))

    model = tf.keras.models.load_model(LSTM_MULTI_MODEL, compile=False)
    return model, tokenizer, max_len

@st.cache_resource
def load_bigru_multilabel():
    with open(BIGRU_TOKENIZER_PATH, "rb") as f:
        tokenizer = pickle.load(f)

    model = tf.keras.models.load_model(BIGRU_MODEL_PATH, compile=False)
    max_len = MAX_LEN_BIGRU
    return model, tokenizer, max_len

# ---- RoBERTa -----

@st.cache_resource
def load_roberta_multilabel():
    tokenizer = AutoTokenizer.from_pretrained(ROBERTA_DIR, local_files_only=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        ROBERTA_DIR,
        local_files_only=True,
    )
    model.to(DEVICE)
    model.eval()
    return tokenizer, model

# 3. Prediction helpers
def predict_binary(text: str, model_choice: str):
    text_clean = preprocess_text(text)

    if model_choice in ["Logistic Regression", "SVM", "Naive Bayes"]:
        model = load_binary_ml_model(model_choice)
        X = [text_clean]

        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X)[0]  # shape (2,) usually
            if len(proba) == 2:
                prob_non = float(proba[0])
                prob_tox = float(proba[1])
            else:
                prob_tox = float(proba[-1])
                prob_non = 1.0 - prob_tox
        else:
            scores = model.decision_function(X)
            score = float(scores[0])
            prob_tox = 1.0 / (1.0 + np.exp(-score))
            prob_non = 1.0 - prob_tox

    elif model_choice == "GRU":
        model, tokenizer, max_len = load_gru_binary()
        seq = tokenizer.texts_to_sequences([text_clean])
        padded = pad_sequences(seq, maxlen=max_len)
        prob_tox = float(model.predict(padded, verbose=0)[0][0])
        prob_non = 1.0 - prob_tox

    else:
        raise ValueError(f"Unknown binary model choice: {model_choice}")

    label = "toxic" if prob_tox >= 0.5 else "non-toxic"
    return label, prob_tox, prob_non


def predict_multilabel(text: str, model_choice: str):
    text_clean = preprocess_text(text)

    # ----- ML models -----
    if model_choice in ["Logistic Regression", "SVM", "Naive Bayes"]:
        model = load_multi_ml_model(model_choice)
        X = [text_clean]

        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X)


            if isinstance(proba, list):
                class_probs = [p[:, 1] for p in proba]  # positive-class prob
                proba_matrix = np.vstack(class_probs).T  # (n_samples, n_labels)
            else:
                proba_matrix = proba  # (n_samples, n_labels)

            probs_sample = proba_matrix[0]
        else:
            scores = model.decision_function(X)  # (1, n_labels)
            probs_sample = 1.0 / (1.0 + np.exp(-scores[0]))

        probs = {label: float(p) for label, p in zip(LABEL_COLS, probs_sample)}

    # ----- LSTM / BiLSTM -----
    elif model_choice == "BiLSTM":
        model, tokenizer, max_len = load_lstm_multilabel()
        seq = tokenizer.texts_to_sequences([text_clean])
        padded = pad_sequences(seq, maxlen=max_len)
        probs_sample = model.predict(padded, verbose=0)[0]  # (6,)
        probs = {label: float(p) for label, p in zip(LABEL_COLS, probs_sample)}

    # ----- BiGRU -----
    elif model_choice == "BiGRU":
        model, tokenizer, max_len = load_bigru_multilabel()
        seq = tokenizer.texts_to_sequences([text_clean])
        padded = pad_sequences(seq, maxlen=max_len)
        probs_sample = model.predict(padded, verbose=0)[0]  # (6,)
        probs = {label: float(p) for label, p in zip(LABEL_COLS, probs_sample)}

    # ----- RoBERTa -----
    elif model_choice == "RoBERTa":
        tokenizer, model = load_roberta_multilabel()
        enc = tokenizer(
            text_clean,
            add_special_tokens=True,
            truncation=True,
            max_length=220,  # same as training
            padding="max_length",
            return_tensors="pt",
        )
        input_ids = enc["input_ids"].to(DEVICE)
        attention_mask = enc["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs_tensor = torch.sigmoid(logits).cpu().numpy()[0]

        probs = {label: float(p) for label, p in zip(LABEL_COLS, probs_tensor)}

    else:
        raise ValueError(f"Unknown multi-label model choice: {model_choice}")

    return probs

Overwriting helpers.py


# **Streamlit app.py**

In [7]:
%%writefile app.py
import streamlit as st
from helpers import LABEL_COLS, predict_binary, predict_multilabel

st.set_page_config(page_title="Toxic Comment Classifier", layout="wide")

st.title("🧪 Toxic Comment Classification Demo")

tab_binary, tab_multi = st.tabs(["Binary Classification", "Multi-label Classification"])

# 1. Binary
with tab_binary:
    st.subheader("Binary Classification (toxic vs non-toxic)")

    binary_model_choice = st.selectbox(
        "Choose a model",
        ["Logistic Regression", "SVM", "Naive Bayes", "GRU"],
        key="binary_model_choice",
    )

    user_input_bin = st.text_area(
        "Enter a comment",
        height=150,
        key="binary_input",
    )

    if st.button("Predict (Binary)", key="predict_binary_btn"):
        if not user_input_bin.strip():
            st.warning("Please enter some text before predicting.")
        else:
            label, prob_tox, prob_non = predict_binary(user_input_bin, binary_model_choice)

            st.markdown(f"**Predicted label:** `{label}`")
            st.write("**Confidence:**")
            st.write(f"- Toxic: `{prob_tox:.4f}`")
            st.write(f"- Non-toxic: `{prob_non:.4f}`")

# 2. Multi-label
with tab_multi:
    st.subheader("Multi-label Classification (6 toxicity categories)")

    multi_model_choice = st.selectbox(
        "Choose a model",
        ["Logistic Regression", "SVM", "Naive Bayes", "BiLSTM", "BiGRU", "RoBERTa"],
        key="multi_model_choice",
    )

    user_input_multi = st.text_area(
        "Enter a comment",
        height=150,
        key="multi_input",
    )

    if st.button("Predict (Multi-label)", key="predict_multi_btn"):
        if not user_input_multi.strip():
            st.warning("Please enter some text before predicting.")
        else:
            probs = predict_multilabel(user_input_multi, multi_model_choice)

            st.write("**Predicted probabilities:**")
            for label in LABEL_COLS:
                st.write(f"- {label}: `{probs[label]:.4f}`")

            st.write("---")
            st.write("**Labels with probability ≥ 0.5:**")
            active_labels = [lbl for lbl, p in probs.items() if p >= 0.5]
            if active_labels:
                st.write(", ".join(active_labels))
            else:
                st.write("_None (all below threshold 0.5)_")

Writing app.py


In [2]:
%pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")

if NGROK_AUTH_TOKEN is None or NGROK_AUTH_TOKEN.strip() == "":
    raise ValueError(
        "NGROK_AUTH_TOKEN secret is empty or not set. "
        "Go to Add-ons → Secrets and create a secret named 'NGROK_AUTH_TOKEN' with your token."
    )

# Set token for pyngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Kill any old tunnels
ngrok.kill()

# Open tunnel to port 8501
public_url = ngrok.connect(8501, "http")
print("🔗 Public URL:", public_url)

🔗 Public URL: NgrokTunnel: "https://unvillainous-subobscurely-larae.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

2025-12-03 15:58:38.478634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764777518.724040     142 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764777518.810336     142 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 16:00:13.972442: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
